In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime

df = pd.read_csv('googleplaystore.csv.csv')

# current time to check 
now = datetime.now()
current_hour = now.hour

# Only show graph between 6 PM and 8 PM  IST
if current_hour >= 18 and current_hour < 20:
    
    # top 5 categories by total installs
    top_5_categories = df.groupby('Category')['Installs'].sum().nlargest(5).index.tolist()
    
    # Filter for only top 5 categories 
    filtered_df = df[df['Category'].isin(top_5_categories)]
    
    # Excluding categories starting with A, C, G, or S
    filtered_df = filtered_df[~filtered_df['Category'].str[0].isin(['A', 'C', 'G', 'S'])]
    
    # Group by Country and Category to get installs
    country_category = filtered_df.groupby(['Country', 'Category'])['Installs'].sum().reset_index()
    
    # Highlight categories where installs exceed 1 million
    country_category['Highlight'] = country_category['Installs'].apply(
        lambda x: 'Over 1M Installs' if x > 1000000 else 'Under 1M Installs'
    )
    
    # creating choropleth map
    fig = px.choropleth(
        country_category,
        locations='Country',
        locationmode='country names',
        color='Installs',
        hover_name='Country',
        hover_data={
            'Category': True,
            'Installs': ':,',
            'Highlight': True
        },
        color_continuous_scale='Viridis',
        title='Global App Installs by Category (Top 5 Categories)',
        labels={'Installs': 'Total Installs'}
    )
    
    # Updating layout for better interactivity
    fig.update_layout(
        geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type='natural earth'
        ),
        height=600,
        title_font_size=16
    )
    
    # Updating color bar
    fig.update_coloraxes(
        colorbar=dict(
            title='Installs',
            tickformat=','
        )
    )
    
    fig.show()
    
    
    print("\nCategories with over 1 million installs:")
    high_install_cats = country_category[country_category['Installs'] > 1000000]
    print(high_install_cats[['Country', 'Category', 'Installs']].to_string(index=False))
    
else:
    print("This graph is only available between 6 PM and 8 PM IST")
    print(f"Current time: {now.strftime('%I:%M %p')}")

This graph is only available between 6 PM and 8 PM IST
Current time: 08:56 AM
